In [1]:
from datetime import datetime
import functools as ft
import gc
import numpy as np
import pickle
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import types, create_engine
from dateutil.relativedelta import relativedelta

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

In [2]:
conn_sql = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=S26;Database=Analytics_WS;Trusted_Connection=yes')

driver = '{ODBC Driver 17 for SQL Server}'
server = 'S28'
db = 'Analytics_WS'
engine_sql = create_engine("mssql+pyodbc:///?odbc_connect=" + 'Driver=%s;Server=%s;Database=%s;QuotedID=Yes;AnsiNPW=Yes;Trusted_Connection=yes' % (driver, server, db))

In [3]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
engine = create_engine(URL(
    account='idb59911.us-west-2',
    user='NLU@NATIONALFUNDING.COM', # change to your username
    authenticator = 'externalbrowser',
    database='SANDBOX',
    role = 'SANDBOX_NLU', # change to your username
    schema='NLU', # change to your username
))
conn = engine.connect()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://nationalfunding.okta.com/app/snowflake/exk8t3c21w5IwCyzl5d7/sso/saml?SAMLRequest=jZLNctowFIVfxaOusWwDCWgwGQcmrQeSEnC66E6xBKjIkqsrY8jTV7ahky6S6U4%2F50jfvedO7k6F9I7cgNAqRqEfII%2BrXDOhdjF6yR56I%2BSBpYpRqRWP0ZkDuptOgBayJEll92rNf1ccrOceUkDaixhVRhFNQQBRtOBAbE42yeOSRH5ASqOtzrVE7yyfOygAN9YRXi0MhMPbW1sSjOu69uu%2Br80OR0EQ4GCMnaqRfLnqT66mD%2FQhDgaN3imc%2FEGbnLdlxciaiiNvdaG9F6prymegr50IyLcsW%2FVW3zcZ8pIr%2FEwrqApuNtwcRc5f1ssOCRyTYK%2FD8TgMfVC63kp64Lkuysq6x3y3wlvOsNQ74TqQzmNUHgR7ThajiN3%2FCm5u9vp5K9PV8Ot%2BmS0KuqiTt5yuq%2BN5sHsa9W2QI%2B%2FHNeKoiTgFqHiqmmCtOwqifi8MeuE4iyIyGJL%2BrR8Oxj%2BRN3fBCkVt67yydnsqt1VbrK8PlraQtCzxX37MT4eR7edRWA%2FTenZ%2Bk0N2iwE0bvJG3QiRFsRM%2F7MNE%2FzedBnCJ5dCOl9pKfJzk19B7cchhX7YngjW27ZSwgsqZMKY4QAuLCl1PTOcWn4ZADztfv132qd%2F

# Parameters

In [4]:
dict_n = {'resp': 8000, 'sub': 8000, 'mail_acq': 2800, 'mail_house':400, 'non_mail': 1000}
additional_n_in_mail = 100
sampled_dfs = {}
universal_cols = ['prospect_type', 'accountnum', 'campaignid', 'lead_key', 'leadid',
       'duns', 'mail_date', 'company', 'address', 'city', 'state', 'zip',
       'flg_type', 'flg_type_grp']
sub_cols = ['Best_CreditReviewId', 'CreatedDate', 'credit_review_ags']

# Collect AGS from the submissions after 2020

In [5]:
sql = """
with dat as
(
select l.id as leadid, l.DUNS_Number__c as duns, la.Lead_AGS, la.Lead_AGS_Tier, credit_review_ags, crc.CreatedDate, o.Best_CreditReviewId
from Salesforce_Repl.dbo.lead l
	INNER JOIN EDW.dbo.Lead_AGS la on la.leadid = l.id
	INNER JOIN [Analytics_DWH].[dbo].[Opportunity_All_VW] AS [o] ON o.AccountId = l.ConvertedAccountId
	INNER JOIN [Salesforce_Repl].[dbo].[Credit_Review__c] AS [crc] ON [crc].[Id] = o.Best_CreditReviewId
where len(DUNS_Number__c)=9 and year(crc.createdDate) >= 2020 and credit_review_ags > 0 
)
select leadid
	, CreatedDate
	, credit_review_ags
	, Best_CreditReviewId
from dat
"""

subs = pd.read_sql(sql, engine_sql)

In [6]:
subs.shape

(33986, 4)

# Response+ Population

In [7]:
sql = """
select rpt.prospect_type
        , rpt.account_number as accountnum
        , rpt.source_campaign_id as campaignid
        , l.lead_key as lead_key
        , l.source_lead_id as leadid
        , coalesce(ar."Duns Number", l.duns_number) as duns
        , to_date(year(rpt.campaignenddate)||'-'||lpad(month(rpt.campaignenddate),2,'0')||'-01') as mail_date
        
        , coalesce(ar.biz, sfl.company) as company
        , coalesce(ar.add, sfl.street) as address
        , coalesce(ar.city, sfl.city) as city
        , coalesce(ar.st, sfl.state) as state
        , coalesce(ar.zip, sfl.postalcode) as zip
        , case when rpt.funding_date is not null then '1_fund'
            when rpt.approved_date is not null then '2_appr'
            when rpt.credit_sub_date is not null then '3_sub'
            when rpt.is_qualified then '4_qual'
            when rpt.response_date is not null then '5_resp' 
            end as flg_type
        , case when rpt.funding_date is not null or rpt.approved_date is not null or rpt.credit_sub_date is not null then 'sub'
            when rpt.is_qualified or rpt.response_date is not null then 'resp' 
            end as flg_type_grp
        , row_number() over (partition by coalesce(ar."Duns Number", l.duns_number) order by l.lead_key) as r
from datawarehouse.marketing.mart_acquisition_performance_report rpt
    left join datawarehouse.core.response_all resp
    on rpt.response_key = resp.response_key
    left join datawarehouse.core.lead l
    on resp.lead_key=l.lead_key
    
    left join dnb.acquisitionmail.snapshot_acqmailfinal_archive ar
    on ltrim(rtrim(ar.accountnum)) = rpt.account_number and ltrim(rtrim(ar.campaignid)) = rpt.source_campaign_id
    left join sourcedata.salesforcenf_marketing.lead sfl
    on sfl.id = l.source_lead_id

where campaignenddate >= '2022-01-01' 
    and campaignenddate < '2023-10-01' 
    and rpt.brand = 'National Funding'
    and rpt.response_date is not null
"""
resp_df = pd.read_sql(sql, engine)

# only pick the ones with duns number
resp_duns = resp_df[(resp_df['r']==1)&(resp_df['duns'].str.len()==9)].copy()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://nationalfunding.okta.com/app/snowflake/exk8t3c21w5IwCyzl5d7/sso/saml?SAMLRequest=jZJbc9owEIX%2Fikd9xrLMJUGDyRBopu7kQsFhOrwJWwYNsuRq5Rj66yvb0EkfkumbLudI3%2B7Zyd2pkN4bNyC0ihDxA%2BRxlepMqH2EXpOH3i3ywDKVMakVj9CZA7qbToAVsqSzyh7Uiv%2BqOFjPPaSAthcRqoyimoEAqljBgdqUrmdPjzT0A1oabXWqJXpn%2BdzBALixjvBqyUA4vIO1JcW4rmu%2F7vva7HEYBAEOxtipGsmXq%2F7kavpAT3AwaPRO4eQP2qS8LStC1lQcecsL7b1QXVM%2BA911IqDfkmTZW76sE%2BTNrvBzraAquFlz8yZS%2Frp67JDAMYlsNxyPCfFB6TqX7MhTXZSVdY%2F5boVznmGp98J1IF5EqDyK7Of9xsyfDuP9Ni6BhduSHFe2yBXfZefk63dG8pfR8sdi01%2FdpsjbXCMOm4hjgIrHqgnWuqMg7PdI0CPjJAzpYETJjT8YkS3yFi5YoZhtnVfWbs9kXrXF%2BvpoWQvJyhL%2F5cf8dLy1%2FTQk9TCu5%2BffcpjdYACNm7xRN0K0BTHT%2F2zDBL83XYbw2aUQL5ZaivTc5Fcw%2B3FIxCftich6eSulvGBCzrLMcAAXlpS6nhvOLL8MAJ52v%2F477dM%2F&R

In [8]:
resp_duns.groupby(['prospect_type', 'flg_type']).agg({'duns':['count','nunique']})

duns        
                         count nunique
prospect_type flg_type                
Acquisition   1_fund      1446    1446
              2_appr      4166    4166
              3_sub       4760    4760
              4_qual     42399   42399
              5_resp    126987  126987
House         1_fund        49      49
              2_appr       151     151
              3_sub        156     156
              4_qual      2131    2131
              5_resp      3478    3478

In [9]:
resp_duns.merge(subs, on=['leadid'], how='left').groupby(['prospect_type', 'flg_type_grp']).agg({'duns':'nunique'})

duns
prospect_type flg_type_grp        
Acquisition   resp          169386
              sub            10372
House         resp            5609
              sub              356

In [10]:
resp_duns.shape
resp_duns['duns'].nunique()

(185723, 15)

185723

In [11]:
for t in ['resp', 'sub']:
    # sampling
    sample_df = resp_duns[resp_duns['flg_type_grp']==t].sort_values(by=['flg_type'])
    sample = sample_df.sample(n=dict_n[t], random_state=999)

    # checking
    sample.shape
    sample['prospect_type'].value_counts()
    sample['prospect_type'].value_counts(normalize=True)
    check_proportion = sample.groupby(['prospect_type', 'flg_type']).agg({'duns':'count'})
    check_proportion['duns'].groupby(level=0).apply(lambda x: x/x.sum())

    # add credit review date
    if t == 'sub':
        sample = sample.merge(subs, on=['leadid'], how='left')
        sample['credit_month'] =  sample['CreatedDate'].dt.date
        sample['mail_date'] = pd.to_datetime(sample['mail_date'],format='%Y-%m-%d').dt.date
        sample['diff'] = (sample['credit_month']- sample['mail_date']).dt.days
        sample['diff'] = np.where(sample['diff'] >= -45, sample['diff'], np.nan)
        sample.sort_values(by=['leadid','diff'], ascending=True, inplace=True)
        sample = sample.groupby(['duns']).head(1) # make sure only 1 record per duns

        sampled_dfs[t] = sample[universal_cols + sub_cols]

    else:
        sampled_dfs[t] = sample[universal_cols]

(8000, 15)

Acquisition    7754
House           246
Name: prospect_type, dtype: int64

Acquisition    0.96925
House          0.03075
Name: prospect_type, dtype: float64

prospect_type  flg_type
Acquisition    4_qual      0.248259
               5_resp      0.751741
House          4_qual      0.394309
               5_resp      0.605691
Name: duns, dtype: float64

(8000, 15)

Acquisition    7735
House           265
Name: prospect_type, dtype: int64

Acquisition    0.966875
House          0.033125
Name: prospect_type, dtype: float64

prospect_type  flg_type
Acquisition    1_fund      0.140530
               2_appr      0.402198
               3_sub       0.457272
House          1_fund      0.135849
               2_appr      0.433962
               3_sub       0.430189
Name: duns, dtype: float64

# mails

In [14]:
for ptc in ['Acquisition', 'House']:
    print(ptc + '...')

    if ptc == 'Acquisition':
        n = dict_n['mail_acq']
        df_label = 'mail_acq'
    else:
        n = dict_n['mail_house']
        df_label = 'mail_house'

    sql  = f"""
    with dat as (
        select distinct 
            Prospect_Type__c "prospect_type"
            , m.Account_Number__c "accountnum"
            , m.CampaignId "campaignid"
            , NULL as lead_key
            , m.leadid
            , coalesce(ar."Duns Number", sfl.duns_number__c) as duns
            , to_date(year(c.enddate)||'-'||lpad(month(c.enddate),2,'0')||'-01') as mail_date    
            , coalesce(ar.biz, sfl.company) as company
            , coalesce(ar.add, sfl.street) as address
            , coalesce(ar.city, sfl.city) as city
            , coalesce(ar.st, sfl.state) as state
            , coalesce(ar.zip, sfl.postalcode) as zip
            ,'6_mail' as flg_type
            ,'mail' as flg_type_grp
            , case when ar."Emp Here" < 2 then 'emp0_1'
                    when ar."Emp Here" < 5 then 'emp2_4'
                    when ar."Emp Here" >= 5 then 'emp5' end as emp
            , case when DATEDIFF('year', to_date(to_char(ar."YEAR")||'-01-01'), enddate) < 1 then 'TIB0'
                    when DATEDIFF('year', to_date(to_char(ar."YEAR")||'-01-01'), enddate) < 6 then 'TIB1_5'
                    when DATEDIFF('year', to_date(to_char(ar."YEAR")||'-01-01'), enddate) >= 6 then 'TIB_6' end as TIB
        from sourcedata.salesforcenf_marketing.campaignmember m
            inner join sourcedata.salesforcenf_marketing.campaign c on m.CampaignId = c.id
            left join dnb.acquisitionmail.snapshot_acqmailfinal_archive ar
                on ltrim(rtrim(ar.accountnum)) = m.Account_Number__c and ltrim(rtrim(ar.campaignid)) = m.CampaignId
            left join sourcedata.salesforcenf_marketing.lead sfl
                on sfl.id = m.leadid

        where enddate >= '2022-01-01' 
            and enddate < '2023-10-01' 
            and c.type = 'Direct Mail'
            and c.Prospect_Type__c in ('{ptc}')
            and try_to_number(coalesce(ar."Duns Number", sfl.duns_number__c)) is not null
        order by m.Account_Number__c
    )
    select *
    from dat SAMPLE row ({n} rows) 
    """
    
    tot_df = pd.DataFrame({'duns':[1, 1]})
    while (tot_df.shape[0] != tot_df['duns'].nunique()):
        tot_df = pd.read_sql(sql, engine)

        # make sure the duns number in mail population is not in response
        tot_df = tot_df[tot_df['duns'].isin(resp_df['duns'])==False].copy() # remove the ones that are already in response
        tot_df = tot_df.sample(n=(n-additional_n_in_mail), random_state=999) # generate a correct sample size
        sampled_dfs[df_label] = tot_df[universal_cols + ['emp', 'tib']]

Acquisition...


prospect_type    2700
accountnum       2700
campaignid       2700
lead_key            0
leadid           2700
duns             2700
mail_date        2700
company          2700
address          2700
city             2700
state            2700
zip              2700
flg_type         2700
flg_type_grp     2700
emp              2687
tib              2652
dtype: int64

House...


prospect_type    300
accountnum       300
campaignid       300
lead_key           0
leadid           300
duns             300
mail_date        300
company          300
address          300
city             300
state            300
zip              300
flg_type         300
flg_type_grp     300
emp                0
tib                0
dtype: int64

prospect_type    300
accountnum       300
campaignid       300
lead_key           0
leadid           300
duns             300
mail_date        300
company          300
address          300
city             300
state            300
zip              300
flg_type         300
flg_type_grp     300
emp                0
tib                0
dtype: int64

In [16]:
sampled_dfs['mail_acq']['emp'].value_counts()
sampled_dfs['mail_acq']['tib'].value_counts()

emp5      1288
emp2_4    1061
emp0_1     338
Name: emp, dtype: int64

In [18]:
sampled_dfs['mail_acq'].head()

,prospect_type,accountnum,campaignid,lead_key,leadid,duns,mail_date,company,address,city,state,zip,flg_type,flg_type_grp,emp,tib
325,Acquisition,7996222227281357,7013n000002QZzwAAG,None,00Q3n00001ZP1oOEAT,102501628,2022-03-01,Klyaksa US LLC,1 Chestnut Pl Apt 1010,Quincy,MA,02169,6_mail,mail,emp5,TIB1_5
359,Acquisition,8226123338680500,7013n000001fVOmAAM,None,00Q3n00001ckL7gEAE,118561378,2023-09-01,"H2u Technologies, Inc.",20360 Plummer St,Chatsworth,CA,91311,6_mail,mail,emp5,TIB1_5
671,Acquisition,8202323334218533,7013n000001fU4zAAE,None,00Q8000001G65d0EAB,058510065,2023-07-01,F C B Trucking LLC,4171 Lomac St,Montgomery,AL,36106,6_mail,mail,emp2_4,TIB_6
1442,Acquisition,8097122240922128,7013n000001NpyNAAS,None,00Q3n00001cmOkqEAE,118528326,2022-09-01,Lockwood's Garden Center,4484 Clark St,Hamburg,NY,14075,6_mail,mail,emp2_4,TIB0
2463,Acquisition,8115222245680746,7013n0000019OJlAAM,None,00Q3400001wpBEFEA2,063402008,2022-11-01,"Barbour Heating & Air, LLC",634 Cobblestone Ln,Stone Mountain,GA,30087,6_mail,mail,emp5,TIB_6


# Non-mails

In [81]:
sql  = f"""
select *
from 
(
    select 
    'Acquisition' as prospect_type,
    NULL as accountnum,
    assg.campaign_id as campaignid,
    NULL as lead_key,
    NULL as leadid,
    ex.duns,
    DATEADD('month', 2, to_date(ex.run_date, 'YYYYMMDD')) as mail_date,
    ex.business_name as company, 
    case when ex.address_2 is null then ex.address 
        else ex.address||' '||ex.address_2 end as address, 
    ex.city, 
    ex.state, 
    ex.zip_code as zip,
    'non_mail' as flg_type,
    'non_mail' as flg_type_grp,
    row_number() over (partition by ex.duns order by ex.run_date) rnk
    from dnb.acquisitionmail.acq_mail_license_export ex
        left join dnb.acquisitionmail.acq_mail_campaign_append app
        on ex.duns=app.duns and app.brand_code='nf' and ex.run_date = app.run_date
        inner join dnb.acquisitionmail.acq_mail_campaign_assignment assg
        on ex.duns=assg.duns and assg.brand_code='nf' and ex.run_date = assg.run_date
        
    where ex.nf_list_name is not null 
        and ex.run_date >= '20221101'
        and ex.run_date <= '20230701'
        and app.duns is null
        and length(ex.address) > 1
)
where rnk = 1
"""
non_mail = pd.read_sql(sql, engine)
non_mail['duns'].count()
non_mail_df = non_mail.sort_values(by=['state'])
sampled_dfs['non_mail'] = non_mail_df[universal_cols].sample(n=dict_n['non_mail'], random_state=997)
sampled_dfs['non_mail'].shape
sampled_dfs['non_mail'].count()

36629

(1000, 14)

prospect_type    1000
accountnum          0
campaignid       1000
lead_key            0
leadid              0
duns             1000
mail_date        1000
company          1000
address          1000
city             1000
state            1000
zip              1000
flg_type         1000
flg_type_grp     1000
dtype: int64

In [82]:
sampled_dfs['non_mail'][sampled_dfs['non_mail']['address'].isna()]

,prospect_type,accountnum,campaignid,lead_key,leadid,duns,mail_date,company,address,city,state,zip,flg_type,flg_type_grp


# Combine all sampled dataframes

In [83]:
sample_final = pd.DataFrame()
for key in sampled_dfs.keys():
    sample_final = pd.concat([sample_final, sampled_dfs[key]], axis=0, ignore_index=True)

# create unique id
sample_final['unique_id'] = (sample_final.index).astype(str).str.zfill(5)
sample_final['run_date'] = sample_final['mail_date']+ relativedelta(months=-2)
sample_final['run_date'] = sample_final['run_date'].astype(str)
sample_final['mail_date'] = sample_final['mail_date'].astype(str)
sample_final.drop(columns=['CreatedDate'], inplace=True)
sample_final.columns = sample_final.columns.str.lower()
# check if we have duplicate duns number
test = sample_final.groupby(['duns'])['prospect_type'].count()
sample_final[sample_final['duns'].isin(test[test >1].index)][['duns','prospect_type','flg_type','flg_type_grp','accountnum','campaignid']].sort_values(by=['duns'])

,duns,prospect_type,flg_type,flg_type_grp,accountnum,campaignid


In [84]:
sample_final.head()

,prospect_type,accountnum,campaignid,lead_key,leadid,duns,mail_date,company,address,city,state,zip,flg_type,flg_type_grp,best_creditreviewid,credit_review_ags,emp,tib,unique_id,run_date
0,Acquisition,8157123322193958,7013n0000019RJLAA2,384ad74ce9ae65cdfd49b798ac3f6b33,00Q3n00001dpCX2EAM,087131029,2023-03-01,Loyal Vending & General Services LLC,12365 SW 255th Ter,Homestead,FL,33032,4_qual,resp,NaN,NaN,NaN,NaN,00000,2023-01-01
1,Acquisition,8185123330938760,7013n0000016DZJAA2,30807d82bedce32d8f7a9912c9a5b71f,00Q3400001p9JaMEAU,080291058,2023-06-01,Trust Multiple Services LLC,3427 Malaga Way,Naples,FL,34105,5_resp,resp,NaN,NaN,NaN,NaN,00001,2023-04-01
2,Acquisition,8085522237408421,7013n000002MNT4AAO,278d13f38b5131dfbdc0de4fbc937ae0,00Q3n00001cm0gUEAQ,052871222,2022-08-01,Legacy Logistics 365 LLC,9478 Garfield Cir,Shreveport,LA,71118,5_resp,resp,NaN,NaN,NaN,NaN,00002,2022-06-01
3,Acquisition,8136422311705501,7013n0000019PqbAAE,9be8515d0d6925e920fb072606b28e00,00Q3n00001b1Y5qEAE,117625838,2023-01-01,"Black Girls Build, LLC",312 Bucknell Cir,Waldorf,MD,20602,5_resp,resp,NaN,NaN,NaN,NaN,00003,2022-11-01
4,Acquisition,7973122225655270,7013n000002QYUPAA4,24c67add0f1d58ccfa7d90a52e9702cc,00Q3n00001Z5bsoEAB,128882772,2022-02-01,Bland Transportation LLC,2741 N 20th St,Philadelphia,PA,19132,5_resp,resp,NaN,NaN,NaN,NaN,00004,2021-12-01


# Saved As Pickcle File

In [85]:
sample_final.to_csv('vendor_test_data_full.csv', index=False, header=True)

In [98]:
sample_final.to_sql('vendor_test_data_full', engine, index=False, if_exists='replace', chunksize=1000)


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://nationalfunding.okta.com/app/snowflake/exk8t3c21w5IwCyzl5d7/sso/saml?SAMLRequest=jZJLc9owFIX%2Fikdd25JNmIAGk6EQWk8TwsNkptkJSyYqsuRKMob8%2Bso2dNJFMt3pcY703Xvu6O5UCO%2FItOFKxiAMEPCYzBTlch%2BDbTr3B8AzlkhKhJIsBmdmwN14ZEghSjyp7Ktcs98VM9ZzD0mD24sYVFpiRQw3WJKCGWwzvJk8PuAoQLjUyqpMCfDO8rmDGMO0dYRXCzXc4b1aW2II67oO6l6g9B5GCCGIhtCpGsmXq%2F7kavpAH0J00%2BidwsnnSmesLSsGVlcMeMsL7Vcuu6Z8BrrrRAZ%2FT9Olv3zapMCbXOGnSpqqYHrD9JFnbLt%2B6JCMY%2BJ01x8OwzAwUtW5IAeWqaKsrHsscCuYMwqF2nPXgWQWg%2FLA6bdFuFi%2F5I%2Bre4pufpKZ2Pzi6fQUrjbHfB7%2BmB6ftuQwuH9OdqsMeM%2FXiKMm4sSYiiWyCda6IxT1%2FBD5US%2BNIty%2Fxf1BMOihF%2BDNXLBcEts6r6zdnoi8aosN1MGSFpKUJfzLD9npMLC9LArrflJPz2%2BiT2%2BhMQo2eYNuhHALosf%2F2YYRfG%2B6DOHCpZDMlkrw7NzkVxD7cUhhELYnnPp5K8WsIFxMKNXMGBeWEKqeakYsuwwAHHe%2F

20000

# vendor version

In [87]:
cols_2_vendor = ['unique_id', 'run_date', 'company', 'address', 'city', 'state', 'zip']
sample_final[cols_2_vendor].to_csv('vendor_test_data.csv', index=False, header=True)

In [89]:
sample_final[cols_2_vendor].nunique()

unique_id    20000
run_date        21
company      19987
address      19821
city          4409
state           53
zip           8632
dtype: int64